## Notebook for checks of RFI masks
### AO, Nov 28, 2023

In [ ]:
import imp
import os
import subprocess
import h5py
import numpy as np
from astropy.time import Time
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import datetime
import matplotlib.dates as mdates
from matplotlib.dates import HourLocator as HourLocator
from matplotlib.dates import MinuteLocator as MinuteLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy import units as u
from astropy.time import TimeDelta

In [ ]:
def read_in_scan(scan_choose):

    directory = '/srv/data/dva/survey_azimuth_scans_Jy_corr/'

    file = h5py.File(directory+'dva_survey_'+scan_choose+'_Jy.h5','r')
    print(file)

    dataset = file['data']['beam_0']['band_SB0']['scan_0']

    freq = file['data']['beam_0']['band_SB0']['frequency'][:]/1e6
    df = freq[1] - freq[0]

    dec   = dataset['metadata']['declination']
    ra    = dataset['metadata']['right_ascension']
    el    = dataset['metadata']['elevation']
    az    = dataset['metadata']['azimuth']
    t     = dataset['metadata']['utc']
    noise = dataset['metadata']['noise_state']
    trim  = dataset['metadata']['trim_scan_flag']

    RR   = dataset['data'][:,0,:]
    LL   = dataset['data'][:,1,:]
    reRL = dataset['data'][:,2,:]
    imRL = dataset['data'][:,3,:]

    PI = np.sqrt((reRL**2)+(imRL**2))
    I = 0.5*(LL+RR)

    noise_idx = np.array(np.where(noise == 1))
    t_plt = Time(t, format='isot',scale='utc').mjd

    PI[noise_idx] = np.nan
    I[noise_idx] = np.nan
    
    mask  = np.load('/srv/data/dva/RFI_mask_full/dva_survey_'+scan_choose+'_RFI_mask.npy')

    I_mask = I.copy()
    I_mask[mask == 1] = np.nan

    PI_mask = PI.copy()
    PI_mask[mask==1] = np.nan
    
    return freq, df, t_plt, I, PI, I_mask, PI_mask

In [ ]:
def waterfall_full():

    fig,axs = plt.subplots(2,2,figsize=(18,18))

    fs = 14

    axs[0,0].imshow(I.T,aspect='auto',vmin=0,vmax=3000,origin='lower',
                    extent=[t_plt[0],t_plt[-1],freq[0],freq[-1]],cmap='viridis')
    axs[0,1].imshow(I_mask.T,aspect='auto',vmin=0,vmax=3000,origin='lower',
                    extent=[t_plt[0],t_plt[-1],freq[0],freq[-1]],cmap='viridis')

    axs[1,0].imshow(PI.T,aspect='auto',vmin=0,vmax=1000,origin='lower',
                    extent=[t_plt[0],t_plt[-1],freq[0],freq[-1]],cmap='viridis')
    axs[1,1].imshow(PI_mask.T,aspect='auto',vmin=0,vmax=1000,origin='lower',
                    extent=[t_plt[0],t_plt[-1],freq[0],freq[-1]],cmap='viridis')

    for i in range(0,2):
        for j in range(0,2):
            axs[i,j].set_xlim(t_plt[0],t_plt[-1])
            axs[i,j].set_ylim(freq[0], freq[-1])
            axs[i,j].tick_params(axis='both', labelsize=fs)
            axs[i,j].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
            axs[i,j].fmt_xdata = mdates.DateFormatter('%H:%M:%S')
            axs[i,j].set_xlabel('Time (UTC)',fontsize=fs)        
            axs[i,j].set_ylabel('Frequency (MHz)',fontsize=fs)
            
    return

In [ ]:
def waterfall_zoom(f_pick,f_band):

    imin = np.where(abs(freq-(f_pick-f_band)) < df/2)[0][0]
    imax = np.where(abs(freq-(f_pick+f_band)) < df/2)[0][0]
    
    print(imin,imax)

    fig,axs = plt.subplots(2,2,figsize=(18,14))
    fs = 14

    axs[0,0].imshow(I[:,imin:imax+1].T,aspect='auto',vmin=0,vmax=3000,origin='lower',
                    extent=[t_plt[0],t_plt[-1],freq[imin],freq[imax]],cmap='viridis',)
    axs[0,1].imshow(I_mask[:,imin:imax+1].T,aspect='auto',vmin=0,vmax=3000,origin='lower',
                    extent=[t_plt[0],t_plt[-1],freq[imin],freq[imax]],cmap='viridis')

    axs[1,0].imshow(PI[:,imin:imax+1].T,aspect='auto',vmin=0,vmax=1000,origin='lower',
                    extent=[t_plt[0],t_plt[-1],freq[imin],freq[imax]],cmap='viridis')
    axs[1,1].imshow(PI_mask[:,imin:imax+1].T,aspect='auto',vmin=0,vmax=1000,origin='lower',
                    extent=[t_plt[0],t_plt[-1],freq[imin],freq[imax]],cmap='viridis')

    for i in range(0,2):
        for j in range(0,2):
            axs[i,j].set_xlim(t_plt[0],t_plt[-1])
            #axs[i,j].set_ylim(freq[imin], freq[imax])
            axs[i,j].tick_params(axis='both', labelsize=fs)
            axs[i,j].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
            axs[i,j].fmt_xdata = mdates.DateFormatter('%H:%M:%S')
            axs[i,j].set_xlabel('Time (UTC)',fontsize=fs)        
            axs[i,j].set_ylabel('Frequency (MHz)',fontsize=fs)
            
    return

In [ ]:
def time_series_plot(f_pick):
    
    fidx = np.where(abs(freq-f_pick) < df/2)[0][0]
    print(fidx,freq[fidx])

    fig,axs = plt.subplots(2,1,figsize=(18,8))
    fs = 14

    axs[0].plot(t_plt,I[:,fidx])
    axs[0].plot(t_plt,I_mask[:,fidx])
    axs[0].set_ylim(0,12000)

    axs[1].plot(t_plt,PI[:,fidx])
    axs[1].plot(t_plt,PI_mask[:,fidx])
    axs[1].set_ylim(0,1000)

    for i in range(0,2):
        axs[i].set_xlim(t_plt[0],t_plt[-1])
        axs[i].tick_params(axis='both', labelsize=fs)
        axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
        axs[i].fmt_xdata = mdates.DateFormatter('%H:%M:%S')
        axs[i].set_xlabel('Time (UTC)',fontsize=fs)        
        #axs[i,j].set_ylabel('Frequency (MHz)',fontsize=fs)
    
    return

In [ ]:
#fig,axs = plt.subplots(1,1,figsize=(18,4))
#axs.plot(freq,I[500,:])
#axs.plot(freq,I_mask[500,:])
#axs.set_ylim(1000,5000)

In [ ]:
###########################
scan_choose = 'phase1_2650'
###########################
freq, df, t_plt, I, PI, I_mask, PI_mask = read_in_scan(scan_choose)
waterfall_full()

In [ ]:
###############
f_pick = 465
f_band = 5
###############
waterfall_zoom(f_pick,f_band)
time_series_plot(f_pick)